In [1]:
!pip install pymysql

  Using cached pymysql-1.1.2-py3-none-any.whl.metadata (4.3 kB)
Using cached pymysql-1.1.2-py3-none-any.whl (45 kB)


In [1]:
import os
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
from datetime import date

In [10]:
API_KEY = "cb3b7ff5870bf1836fcc688dc62dcb4f"
CITY = "Pune"

In [3]:
USER = "root"
PASS = "Root"
HOST = "127.0.0.1"
PORT = 3306
DB = "sales_weatherinfo_db"

In [4]:
MYSQL_CONN_STRING = f'mysql+pymysql://{USER}:{quote_plus(PASS)}@{HOST}:{PORT}/{DB}'

In [5]:
def fetch_weather(api_key: str, city: str) -> dict:
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": api_key.strip(), "units": "metric"}

    r = requests.get(url, params=params, timeout=20)
    if r.status_code != 200:   # ❌ Fixed status code check
        try:
            print("OpenWeather error payload:", r.json())
        except Exception:
            print("OpenWeather non-JSON response:", r.text)
        r.raise_for_status()

    data = r.json()
    return {
        "weather_data": date.today(),  # ✅ Correct key name
        "city": city,
        "temp_c": data["main"]["temp"],  # ✅ Correct data field for temperature
        "humidity": data["main"]["humidity"],  # ✅ Add humidity if it's in the table
        "description": data["weather"][0]["description"],
    }


In [6]:
def ensure_weather_table(engine) -> None:
    ddl="""
    CREATE TABLE IF NOT EXISTS weather(
    weather_data DATE NOT NULL,
    city VARCHAR(100) NOT NULL,
    temp_c DECIMAL(5,2),
    humidity INT,
    description VARCHAR(255)
    )
    """

    with engine.begin() as conn:
        conn.execute(text(ddl))

In [11]:
def store_weather_to_db(weather_data: dict,conn_string:str) -> None:
    engine = create_engine(conn_string,pool_pre_ping=True)
    ensure_weather_table(engine)
    try:
        df = pd.DataFrame([weather_data])
        df.to_sql("weather",engine,if_exists="append",index=False)
        print(f"Weather data for {weather_data['city']} stored successfully in MySQL.")
    finally:
        engine.dispose()

In [12]:
def main():
    weather=fetch_weather(API_KEY,CITY)
    store_weather_to_db(weather,MYSQL_CONN_STRING)

In [13]:
if __name__=="__main__":
    main()

Weather data for Pune stored successfully in MySQL.
